## 1 - Trigger 

### Enunciado

Crear trigger que registre, en una tabla de monitoreo, cada vez que un producto supere las 200.000 unidades de ventas acumuladas.
El trigger debe activarse después de registrada una nueva venta e insertar en la tabla de monitoreo ID de producto, nombre, nueva cantidad total ventas y la fecha en la que se superaron los 200k vendidos.
  
---

### Tabla de monitoreo

```sql
CREATE TABLE product_monitoring (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    ProductID INT,
    ProductName VARCHAR(250),
    ProductTotalSold INT,
    ThresholdDate DATETIME DEFAULT CURRENT_TIMESTAMP
);
```

### Trigger

```sql
DELIMITER $$

CREATE TRIGGER trigger_product_sold_threshold_200k
AFTER INSERT ON sales
FOR EACH ROW
BEGIN
    DECLARE product_total_sold_sales INT DEFAULT 0;

    SELECT SUM(Quantity)
    INTO product_total_sold_sales
    FROM sales
    WHERE ProductID = NEW.ProductID;

    IF product_total_sold_sales > 200000 THEN
        IF NOT EXISTS (
            SELECT 1 FROM product_monitoring
            WHERE ProductID = NEW.ProductID
        ) THEN
            INSERT INTO product_monitoring (
                ProductID,
                ProductName,
                ProductTotalSold
            )
            SELECT 
                p.ProductID,
                p.ProductName,
                product_total_sold_sales
            FROM products p
            WHERE p.ProductID = NEW.ProductID;
        END IF;
    END IF;
END$$

DELIMITER;
```

## Registro de venta para accionar trigger

Se registra una venta de manera que se accione el trigger.

```sql
INSERT INTO sales (SalesID, SalesPersonID, CustomerID, ProductID, Quantity, TotalPrice, SalesDate)
SELECT 
    COALESCE(MAX(salesID), 0) + 1,
    9,
    84,
    103,
    1876,
    1200,
    '2025-06-12 16:03:30.270'
FROM sales;
```

Se consulta la tabla de monitoreo.

```sql
select *
from product_monitoring;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej4-2.png)

Cometarios

- La venta insertada hace que el acumulado de ventas del producto con ID 103 superara el umbral de 200k unidades. 
- Superadas las 200k en ventas acumuladas, se activa el trigger trigger_product_sold_threshold_200k. 
- La ejecución del trigger inserta en la tabla product_monitoring los datos necesarios.



## 2 - Optimización de consultas mediante índices

### Enunciado

Selecciona dos consultas del avance 1 y crea los índices que consideres más adecuados para optimizar su ejecución.

Se deben probar tanto índices individuales como compuestos, de acuerdo a la lógica de cada consulta.  
Ejecutar nuevamente ambas consultas para comparar los tiempos antes y después de aplicar los índices.  

---

### Consulta 1. Productos más vendidos y su vendedor principal

Query

```sql
WITH 
top5_selling_products AS (
    SELECT
        pr.ProductID,
        pr.ProductName,
        SUM(s.Quantity) AS total_quantity
    FROM sales s
    JOIN products pr ON pr.ProductID = s.ProductID
    GROUP by pr.ProductID, pr.ProductName
    ORDER by total_quantity DESC
    LIMIT 5
),
sales_with_seller_name AS (
    SELECT 
        s.ProductID,
        s.SalesPersonID,
        s.Quantity,
        CONCAT(e.FirstName, ' ', e.LastName) AS seller_name
    FROM sales s
    JOIN employees e ON s.SalesPersonID = e.EmployeeID
),
sales_by_seller_and_product AS (
    SELECT
        s.ProductID, 
        s.SalesPersonID, 
        SUM(s.Quantity) AS quantity_sold,
        s.seller_name
    FROM sales_with_seller_name s
    GROUP BY s.ProductID, s.SalesPersonID, s.seller_name
),
top_seller_per_product AS (
    SELECT 
        ssp.ProductID, 
        ssp.SalesPersonID, 
        ssp.quantity_sold, 
        ssp.seller_name
    FROM 
        sales_by_seller_and_product ssp
    JOIN (
        SELECT 
            ProductID, 
            MAX(quantity_sold) AS max_sold
        FROM sales_by_seller_and_product 
        GROUP BY ProductID
    	) t1 ON ssp.ProductID = t1.ProductID AND ssp.quantity_sold = t1.max_sold
)
SELECT 
    t5sp.ProductID AS product_id,
    t5sp.ProductName AS product_name, 
    tspv.SalesPersonID AS sales_person_id, 
    tspv.seller_name, 
    tspv.quantity_sold AS quantity_sold_by_seller,
    t5sp.total_quantity AS total_quantity_sold  
FROM top5_selling_products t5sp
JOIN top_seller_per_product tspv ON tspv.ProductID = t5sp.ProductID
ORDER BY t5sp.total_quantity DESC;
```

Indices

El uso de los índices, detallados a continuación, reducen en aproxidamente un 80% el tiempo de ejecución.

```sql
CREATE INDEX idx_sales_productID_salesPersonID_quantity
ON sales (ProductID, SalesPersonID, Quantity);
```

- Optimiza el JOIN con products usando ProductID.

- Optimiza el JOIN con employees usando SalesPersonID.

- Mejora el cálculo de la suma SUM(Quantity) en el grupo por producto y vendedor.

- La presencia de Quantity en el índice hace que se pueda cubrir la consulta sin tener que leer la tabla completa.


```sql
CREATE INDEX idx_products_productID_productName
ON products (ProductID, ProductName);
```

- Optimiza el JOIN con sales usando ProductID.

- No se requieren lecturas adicionales para obtener ProductName ya que está incluido en el índice.

---

### Consulta 2. Adopción productos más vendidos 

Query

```sql
WITH top5_products AS (
    SELECT ProductID, SUM(Quantity) AS total_quantity
    FROM sales
    GROUP BY ProductID
    ORDER BY total_quantity DESC
    LIMIT 5
),
unique_customers_per_product AS (
    SELECT t5p.ProductID, COUNT(DISTINCT s.CustomerID) AS unique_customers
    FROM top5_products t5p
    JOIN sales s ON t5p.ProductID = s.ProductID
    GROUP BY t5p.ProductID
),
total_customers AS (
    SELECT COUNT(DISTINCT CustomerID) AS total
    FROM sales
)
SELECT 
    t5p.ProductID as product_id,
    pr.ProductName as product_name,
    t5p.total_quantity as total_quantity_sold,
    ucpp.unique_customers,
    tc.total AS total_customers,
    ROUND(ucpp.unique_customers / tc.total * 100, 2) AS customer_percentage
FROM top5_products t5p
JOIN unique_customers_per_product ucpp ON t5p.ProductID = ucpp.ProductID
JOIN total_customers tc
JOIN products pr ON t5p.ProductID = pr.ProductID
ORDER BY ucpp.unique_customers DESC;
```

Indices

El uso de los índices, detallados a continuación, reduce en más de un 75% el tiempo de ejecución.

```sql
CREATE INDEX idx_sales_product_customer_quantity ON sales(ProductID, CustomerID, Quantity);
```

- Optimiza el cálculo del total vendido (SUM(Quantity))

- Mejora el conteo de  clientes únicos (COUNT(DISTINCT CustomerID))

- Optimiza la unión con la tabla products (JOIN con ProductID)

Este índice contiene todas las columnas necesarias para resolver la consulta, lo que reduce el tiempo de lectura de datos desde la tabla y mejora el rendimiento de la mema.


```sql
CREATE INDEX idx_products_productID_productName
ON products (ProductID, ProductName);
```

- Optimiza el JOIN con sales usando ProductID.

- No se requieren lecturas adicionales para obtener ProductName ya que está incluido en el índice.

